# ZeRO (Zero Redundancy Optimizer)

## [1.ZeRO-DP](https://arxiv.org/pdf/1910.02054)

В то время как существующий подход DP воспроизводит состояния модели на каждом устройстве и создает значительную нагрузку на память, ZeRO-DP устраняет эту избыточность памяти, разделяя их — состояния оптимизатора, градиенты и параметры — между параллельными процессами данных. На рисунке 1 показана количественная и визуализированная потребность в памяти с ZeRO-DP и без него. На рисунке показан объем памяти, занимаемый после разбиения (1) состояния оптимизатора, (2) градиента и (3) избыточности параметров. Они обозначаются: $P_{os}, P_g и P_p$ соответственно.

**Замечание:**
1. Optimizer State Partitioning ($P_{os}$): ZeRO-1
2. (1) + Gradient Partitioning ($P_{os+g}$): ZeRO-2
3. (2) + Parameter Partitioning ($P_{os+g+p}$): ZeRO-3

![ZeRO-optiizer](../Pictures/ZeRO/ZeRO-optimizer.png)

**Пояснение:** $Ψ$ обозначает размер модели (количество параметров), $K$ обозначает множитель памяти состояний оптимизатора, а $N_d$ обозначает степень DP. В примере мы предполагаем размер модели $Ψ = 7,5B$ и DP $N_d = 64$ с $K = 12$ на основе обучения со смешанной точностью с помощью оптимизатора Адама.

**Замечание:** [Статья, которая очень подробно рассказывает, откуда берутся значения и про некоторые оптимизации](https://huggingface.co/blog/Isayoften/optimization-rush)

### 1.1.  $P_{os}$: Optimizer State Partitioning

Для DP со степенью $N_d$ мы группируем состояния оптимизатора в равные $N_d$ разделы, так что i-й процесс обновляет только те состояния оптимизатора, которые соответствуют i-му разделу. Таким образом, каждый параллельный процесс должен хранить и обновлять только $\frac{1}{N_d}$ от общего числа состояний оптимизатора, а затем обновлять только $\frac{1}{N_d}$ параметров. В конце каждого этапа обучения мы выполняем сбор данных по всему параллельному процессу обработки данных, чтобы получить полностью обновленные параметры для всего параллельного процесса обработки данных

**Экономия памяти:** Потребление памяти после оптимизации раздела состояния уменьшается с $4Ψ + KΨ$ до $4Ψ + \frac{KΨ}{Nd}$.

### 1.2. $P_g$: Gradient Partitioning

Поскольку каждый параллельный процесс обработки данных обновляет только соответствующую секцию параметров, ему требуются только уменьшенные градиенты для соответствующих параметров. Следовательно, по мере того, как каждый градиент каждого слоя становится доступным во время обратного распространения, мы уменьшаем их только на параллельном процессе данных, отвечающем за обновление соответствующих параметров. После редукции градиенты нам больше не нужны, и память о них может быть освобождена. Это уменьшает объем памяти, необходимый для хранения градиентов от $2Ψ$ байт до $\frac{2Ψ}{N_d}$. По сути, это операция Reduce-Scatter, при которой градиенты, соответствующие различным параметрам, сводятся к другому процессу. Чтобы сделать это более эффективным на практике, мы используем стратегию бакетизации, при которой мы ведем все градиенты, соответствующие конкретному разделу, и выполняем редукцию сразу для всего бакета.([Что-то похожее есть в PyTorch](https://pytorch.org/docs/stable/notes/ddp.html))

### 1.3. $P_p$: Parameter Partitioning

Как и в случае с состояниями оптимизатора и градиентами, каждый процесс хранит только параметры, соответствующие его разделу. Когда параметры за пределами его раздела требуются для прямого и обратного распространения, они поступают от соответствующего параллельного процесса данных посредством широковещательной рассылки. Хотя на первый взгляд может показаться, что это влечет за собой значительные накладные расходы на коммуникацию, мы показываем, что такой подход увеличивает общий объем связи базовой системы DP только в 1,5 раза, обеспечивая при этом уменьшение памяти пропорционально $N_d$

Экономия памяти: С помощью разбиения параметров мы уменьшаем потребление памяти моделью параметров $Ψ$ с $16Ψ$ до $\frac{16Ψ}{N_d}$.



## [2.ZeRO-R](https://arxiv.org/pdf/1910.02054)

## 2.1.$C_B$: Constant Size Buffers 

ZeRO тщательно выбирает размеры буферов временных данных, чтобы сбалансировать память и эффективность вычислений. Во время обучения вычислительная эффективность некоторых операций может сильно зависеть от размера входных данных, при этом большие входные данные достигают более высокой эффективности. Например, большая операция all-reduce обеспечивает гораздо более высокую пропускную способность, чем меньшая. Следовательно, для повышения эффективности высокопроизводительные библиотеки, такие как NVIDIA Apex или Megatron, объединяют все параметры в один буфер перед применением этих операций. Однако накладные расходы на память объединенных буферов пропорциональны размеру модели и могут стать тормозящими. Например, для модели параметров 3B для 32-разрядного буфера с плавлением потребуется 12 ГБ памяти. Чтобы решить эту проблему, мы просто используем эффективный по производительности буфер с постоянным размером, когда модель становится слишком большой. Таким образом, размер буфера не зависит от размера модели, и, сохраняя размер буфера достаточно большим, мы все еще можем достичь хорошей эффективности

## 2.2.$M_D$: Memory Defragmentation

Фрагментация памяти при обучении модели происходит в результате активации контрольных точек и градиентных вычислений. Во время прямого распространения с контрольными точками активации только выбранные активации сохраняются для обратного распространения, в то время как большинство активаций отбрасывается, так как они могут быть повторно вычислены во время обратного распространения. Это создает чередование короткой памяти (отброшенные активации) и долгоживущей памяти (активация с контрольной точкой), что приводит к фрагментации памяти. Аналогичным образом, при обратном распространении градиенты параметров являются долгоживущими, в то время как градиенты активации и любые другие буферы, необходимые для вычисления градиентов параметров, являются кратковременными. Опять же, это чередование кратковременной и долговременной памяти приводит к фрагментации памяти.

Ограниченная фрагментация памяти обычно не является проблемой, когда памяти много, но для обучения больших моделей, работающих с ограниченной памятью, фрагментация памяти приводит к двум проблемам: i) OOM из-за недостатка непрерывной памяти даже при достаточном объеме доступной памяти, ii) низкой эффективности в результате того, что распределитель памяти тратит значительное время на поиск непрерывного фрагмента памяти для удовлетворения запроса памяти. ZeRO выполняет дефрагментацию памяти на лету, предварительно выделяя непрерывные блоки памяти для контрольных точек активации и градиентов и копируя их в предварительно выделенную память по мере их создания. MD не только позволяет ZeRO обучать более крупные модели с большими размерами партий, но и повышает эффективность при обучении с ограниченной памятью.

## [3. ZeRO-infinity](https://arxiv.org/pdf/2104.07857)

ZeRO-Infinity построен на основе ZeRO-3, он разделяет все состояния модели для устранения избыточности памяти. В отличие от любой из существующих технологий семейства ZeRO, ZeRO-Infinity разработан с мощным механизмом разгрузки, называемым механизмом infinity offload, который может выгружать все разделенные состояния модели на CPU или NVMe память или сохранять их на GPU в зависимости от требований к памяти. 

Снапшот общего дизайна ZeRO-infinity

![Snapshot](../Pictures/ZeRO/Zero-infinity-Snapshot.png)

### 3.1. Memory-centric tiling for working memory

Чтобы снизить требования к рабочей памяти для обучения DL для больших моделей, ZeRO-Infinity представляет новую технику, называемую тайлингом, ориентированным на память, которая использует шаблон выборки и отпускания данных ZeRO-3 для снижения требований к рабочей памяти путем разбиения большого оператора на более мелкие листы, которые могут выполняться последовательно. Например, чтобы уменьшить рабочую память для большого линейного оператора, ZeRO-Infinity представляет оператор в виде математически эквивалентной последовательности меньших линейных операторов, состоящей из плиток параметров исходного оператора, и выполняет их последовательно. В сочетании с ZeRO-3 параметр и градиенты каждой плитки можно извлекать и отпускать по одному, уменьшая рабочую память пропорционально количеству плиток. Таким образом, ZeRO-Infinity может поддерживать операторы произвольных размеров, не полагаясь на параллелизм моделей для их размещения в ограниченной памяти графического процессора

### 3.2. Bandwidth-Centric Partitioning

ZeRO-Infinity разделяет отдельные параметры по всем параллельным процессам данных и использует allgather вместо широковещательной рассылки, когда необходимо получить доступ к параметру. Обратите внимание, что коллективы связи broadcast и allgather имеют одинаковые затраты на связь, когда речь идет об объеме перемещения данных, если данные расположены на графическом процессоре. Таким образом, это не имеет никакого значения для обучения только на GPU. Однако это меняет правила игры, когда данные находятся в NVMe или CPU.

В подходе, основанном на широковещательной передаче, поскольку каждый параметр полностью принадлежит одному из параллельных процессов передачи данных, параметр должен быть сначала передан из исходного местоположения в память графического процессора через PCIe, прежде чем может произойти широковещательная передача. Обратите внимание, что для этого процесса может быть активен только один PCIe, в то время как все каналы PCIe, подключенные ко всем остальным графическим процессорам, простаивают. Напротив, при использовании секционированного параметра и подхода, основанного на allgather в ZeRO-Infinity, все каналы PCIe активны параллельно, каждый из них вносит 1/dp части параметра, где dp — это степень параллельности данных. В результате, эффективная пропускная способность связи между NVMe или CPU и графическим процессором линейно увеличивается со степенью dp.

### 3.3. Overlap Centric Design

В то время как ZeRO-Infinity может использовать достаточную пропускную способность гетерогенной памяти в конфигурации с несколькими узлами, пропускная способность все еще может быть узким местом в конфигурации с одним графическим процессором или одним узлом. Даже связь GPU-GPU allgather оказывает большое влияние на эффективность при работе с небольшим размером партии. Кроме того, доступ к памяти NVMe требует трехэтапного процесса: 
 1. чтение данных из NVMe в память CPU (nc-transfer)
 3. копирование данных из памяти CPU в память GPU (cg-transfer)
 3. выполнение allgather для создания полного параметра на всех графических процессорах (gg-transfer).

Последовательный характер этих перемещений данных означает, что при наивном подходе общее время связи будет равным сумме затрат на перемещение данных по каждому из этих трех этапов, что приведет к низкой эффективности, даже если пропускная способность для перемещения данных на каждом из этих этапов достаточна по отдельности.

Чтобы решить эти проблемы, ZeRO-Infinity имеет механизм перекрытия, который не только перекрывает взаимодействие GPU-GPU с вычислениями GPU, но также перекрывает взаимодействие NVMe с CPU и CPU, и все это одновременно

Механизм перекрытия состоит из двух компонентов:
 1. динамическая предварительная выборка для перекрытия перемещения данных, необходимого для восстановления параметров до того, как они будут использованы при прямом или обратном проходе
 2. механизм связи и разгрузки перекрытия для выполнения перемещения данных, необходимого для градиентов, параллельно с обратными вычислениями.

Динамическая предварительная выборка в ZeRO-Infinity отслеживает прямые и обратные вычисления на лету, создавая внутреннюю карту последовательности операторов для каждой итерации. Во время каждой итерации предварительная выборка отслеживает, где она находится в последовательности операторов, и выполняет предварительную выборку параметра, требуемого будущими операторами. Предварительная выборка знает о трехступенчатом процессе взаимодействия и поэтому может перекрывать nc- transger для одного параметра с cg-transfer и gg-transfer других параметров. Например, перед выполнением $i$-го оператора, оператор предварительной выборки может вызвать nc, cg и gg-transfer для параметров, требуемых операторами $i + 3$, $i + 2$ и $i + 1$ соответственно. Обратите внимание, что все эти перемещения данных могут происходить параллельно с выполнением оператора i. Кроме того, ZeRO-Infinity может обновлять карту последовательностей операторов в случае динамического рабочего процесса, обеспечивая соответствующую предварительную выборку даже при изменении прямого и обратного распространения между итерациями.

### 3.4. Infinity Offload Engine
[DeepNVMe](https://github.com/deepspeedai/DeepSpeed/blob/master/blogs/deepspeed-gds/README.md) - поддерживает массовые запросы на чтение и запись для асинхронного завершения и явные запросы синхронизации для сброса текущих операций чтения и записи. Поддержка асинхронности позволяет ZeRO-Infinity перекрывать эти запросы с помощью связи или вычислений между GPU/GPU или GPU/CPU.



## [4. ZeRO-inference](https://www.deepspeed.ai/2022/09/09/zero-inference.html)

ZeRO-Inference закрепляет веса всей модели в CPU или NVMe (в зависимости от того, что достаточно для размещения полной модели) и передает веса слой за слоем в графический процессор для вычисления вывода. После вычисления слоя выходные данные сохраняются в памяти графического процессора в качестве входных данных для следующего слоя, в то время как память, потребляемая весами слоев, освобождается для использования следующим слоем. Таким образом, время вывода модели состоит из времени вычисления слоев на GPU и времени на выборку слоев через PCIe. Для вывода больших моделей этот подход обеспечивает преимущества масштабирования и эффективности. На рисунке ниже показан максимальный размер модели, который можно использовать на видеокарте Nvidia V100

![V100](../Pictures/ZeRO/Nvidia-V100.png)

**Используемые оптимизации:**

 1. Overlap-centric design (на 1 или нескольких GPU)
 2. Bandwidth-Centric Partitioning (на нескольких GPU)


### Экспериенты и выводы по использованию на одной видеокарте с трансформерами

#### Полная выгрузка и частичная выгрузка весов модели

Ключевым выбором в ZeRO-Offload является разгрузка всех весов моделей, размер которых превышает память графического процессора, а не размещение подмножества весов в памяти графического процессора. В представлены результаты генерации токенов OPT-30B на одном устройстве V100-32GB, в которых сравнивается полная разгрузка весов модели и размещение части (т. е. 10 и 12 миллиардов параметров) в памяти графического процессора. Результаты показывают, что полная разгрузка обеспечивает наилучшую производительность как для памяти ЦПУ (43 маркера в секунду), так и для памяти NVMe (30 маркеров в секунду). Как с процессором, так и с памятью NVMe полная разгрузка происходит более чем в 1,3 и 2,4 раза быстрее, чем частичная разгрузка 18 и 20 миллиардов параметров соответственно. Преимущество в производительности при полной разгрузке заключается в больших размерах партии по сравнению с вариантами частичной разгрузки. Таким образом, когда модель не помещается в GPU, использование памяти GPU для увеличения размера пакета, а не для частичного размещения модели, приводит к более быстрой генерации токенов.

![Partial-and-fully-offload](../Pictures/ZeRO/Partial-and-fully-offload.png)

#### Предварительная выборка веса слоев

ZeRO-Inference извлекает слои перед использованием, перекрывая их с текущими вычислениями слоев, чтобы скрыть задержку передачи слоев. Мы измеряем влияние предварительной выборки на производительность генерации токенов на одном устройстве V100-32GB и обобщаем результаты в таблице ниже. Мы видим, что предварительная выборка не улучшила разгрузку процессора. Это связано с тем, что относительно короткие последовательности генерации токенов (т. е. менее 50 токенов) привели к тому, что время вычисления слоя было недостаточным для того, чтобы скрыть значительную часть времени выборки слоя от процессора. В отличие от этого, предварительная выборка повышает производительность разгрузки NVMe в 1,13 раза, 1,14 и 1,21 раза для OPT-30B, OPT-175B и BLOOM-176B соответственно. Это связано с тем, что передача весов из NVMe через память ЦП позволяет выполнять предварительную выборку для перекрытия передач из ЦП в память графического процессора с передачами из NVMe в ЦП, увеличивая эффективную пропускную способность

![Prefecting-layer-weights](../Pictures/ZeRO/Prefetching-layer-weights.png)

ZeRO-Inference использует четыре межкомпонентных соединения PCIe между графическими процессорами и памятью процессора для распараллеливания выборки слоев для более быстрых вычислений вывода на нескольких графических процессорах. В таблице ниже мы сообщаем об увеличении пропускной способности при генерации токенов на двух и четырех графических процессорах по сравнению с одним графическим процессором. Эти результаты были собраны с включенной предварительной выборкой слоев. Сообщаемые значения пропускной способности относятся к графическому процессору, показывая, что генерация маркеров становится быстрее на каждом графическом процессоре, поскольку агрегированные каналы PCIe уменьшают задержки при выборке слоев. Повышенная пропускная способность на графический процессор приводит к сверхлинейному масштабированию. Кроме того, эти результаты свидетельствуют о том, что улучшенная пропускная способность будущих поколений PCIe может помочь повысить производительность ZeRO-Inference.

![Parallelizing layer fetching on multiple GPUs](../Pictures/ZeRO/Parallelizing%20layer%20fetching%20on%20multiple%20GPUs.png)
